In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/tarin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with io.open('short_text.txt', encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

corpus length: 553992


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 872


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 184651
Vectorization...


In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [8]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
184651/184651 [==============================] - 216s 1ms/step - loss: 3.5183

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "成けり中將のさらにむかし
ざまのにほひにもみえぬならひに。さしもにぬ
ものとおも"
成けり中將のさらにむかし
ざまのにほひにもみえぬならひに。さしもにぬ
ものとおもひしりて。いとゞかくおぼしたる御
かたのみなりける。さま＋＋＄にて。いとおしく。
かくてなどもおぼしたる御かたの御こゝろに
さま＋＋＄にて。かくらひ給て。かくおもひのかた
なりけり。いとおしくおぼすかなりけり。いと
おほくものゝしらず。いかにおもひしるを。
かくおはしますかの御こゝろなどのみえ給。
かなしの御ことをも。おほかりけり。いとゞもしく
かくおぼするを。いとおしくもの給ふかうなり
かしの御かたのみなど。いとゞめきたる御心ばへの
かくなども。いとおしくものし給へば。いと
かくおぼすかならず。いとゞもしくおぼしかし
からず。いとゞめかしくもおぼされて。いとゞかく
おぼしかなしきこともありけり。おほくかならず
なりけり。さるまじきことに。いとおしくもの
かしのみなどのみえ給ふべきことは。この御ことに
なりとしりて。かくらうたくもおぼしたる御
かななども。いとおしく。いとゞめしきことなり。
----- diversity: 0.5
----- Generating with seed: "成けり中將のさらにむかし
ざまのにほひにもみえぬならひに。さしもにぬ
ものとおも"
成けり中將のさらにむかし
ざまのにほひにもみえぬならひに。さしもにぬ
ものとおもひしるべきことに。おほくものしたり。
かなきことをも。ひき＋＋＄にもからひ給て。いといと
おしくもの給へばかりなり給て。おほくし
かしうみいたる御かたのしを。ある人もとの
さまにかくし給ふ。あはれにおもひつるに。おもひ
つかくいとおしく。おぼしたるを。たゞこの御心
には。すこしのとをこそあらず。いまはらかしければ
との給ふかうのことありける。いとはしきこと
なれば。人やうておほくの給へば。おもふかたの
ほどに。ひ

たてこなもまつ物すがらに。ろ
やすくかたへの物をしばかりいふ
けしかりなれどに世に
あらまち。し御らん
ありける程よりし御
女ざのかのかのらみだにかく
成にしありたるとは
などおそろしと。
てなきることもうれな
かるるをまた
さまにゐ給ふはなじえも
うちと身の御いりはりかの臺にときなど
こせたらずありけるむかたにつけて。いかに
わたりての給ふこのいとにはあれど大殿のにもふの
つゐもかのちのまたいなうざるなど思ひきこ
なるくり。はづかしくもづからをきこせかどきみ
はねいひけりろかなどらうたまさまほしきことさ
の聞え給へはらこなかよしさて人にいでやから
なん大将ののづから御此をきね＋＋に心あくが
ふふるゝ。人々＄のかひゐてものし給
----- diversity: 1.2
----- Generating with seed: "おもゝちよにみえぬさまなり詠など
し給へるは是や佛の御迦陵■伽のこゑならんと
き"
おもゝちよにみえぬさまなり詠など
し給へるは是や佛の御迦陵■伽のこゑならんと
きごゆゝろうつき人に世をしすてゝくば心
をこはらかひみげんこそいととけじ。さま＄をくら
むともみに有ける御けしき侍れ。の女る＋ごも奉れ
てこはながめおぼえやかみわま心のをしふして。
御らんが御人々みそそしき物はは。うた。世に人やさ
らうにて。かんはすなれ心のあこまし給ける。ゆう
かゝせさせてもかかりきあかりに。うつをのすゑのいと
よすうゆへしなことにいつかき。花やはたちをはやれぬは
すつかばこいなれるべきはびんそよれん出もとな
らすにそゐにもせさせ給へらんよとありたとおぼだ
世中かたわざとて
う身々はしみけり。夜ながさ
りなる御ゆがらにけざめのそう
さゝめげて日さふ
かつ＋＋まこ
けどかんにはて宮の
わたりのくはとより。え。さく
らうものう。ぎにさだりき世のゝへ
わき給へる。
せさせいで葉に宮しもおぼつかめ
かはするよせばこるしざめとなろ
ひみゆきぎたればいの
奉りていとずりは
おはしぬさま
Epoch 5/60
184651/184651 [==============================] - 216s 1ms/step - loss: 2.8616

----- Generating text after Epoch: 4
----- diversity: 0.2
-

ことを。いとゞめたまへる御ありさま。いとゞおもし
きこえ給て。いとゞあやしくかくおぼしたり。ことに
おぼす。かくおぼしたちて。かゝる御こゝろをもり
などし給ふに。かゝる御こゝろにもあらん。いと
おもしろしきこえさせ給て。いとゞおほかたの
----- diversity: 0.5
----- Generating with seed: "しない給てこなたはみづ
むまやなりけれど。けはひにぎはゝしく。人々＄こゝろ
げさ"
しない給てこなたはみづ
むまやなりけれど。けはひにぎはゝしく。人々＄こゝろ
げさきたる御ありさま。あやしく。けふ人のみは
いとながらぬるに。いとよくなり給へることをこそ
ありけり。かくしもしらずなどし給に。おほかた
おぼして。いとわびしからず。いとかたじけなく
すぐし給もにくころにつくし給ふ御こゝろに
かへりて。いとおしうおぼしわたるを。ことにかく
おぼしあはするにこのかくなんとおぼす。いとゞ
まかくて。かたらひ給て。かゝる御ありさま。あながち
なきに。いとゞあさましきことに。きゝわたりて
さすがにおぼしなげくなりしにもあらず。いと
おかしげなる御こゝろもなしのうちにもの給ふ
御こゝろにくちおしき御こゝろにもあらず。なにごと
をもまかせて。かゝるまでものし給に。わたり給ふ。
さだめありしなど。人のみありて。うちかすき
たちかへり給へる御心もあるかな。かゝることを
思ひつゞくらせ給へば御心をもてうちなげき
給へる御心にて御なからひにいできて。たがふ
ことを。かゝる御すく
----- diversity: 1.0
----- Generating with seed: "しない給てこなたはみづ
むまやなりけれど。けはひにぎはゝしく。人々＄こゝろ
げさ"
しない給てこなたはみづ
むまやなりけれど。けはひにぎはゝしく。人々＄こゝろ
げさはれたるやうにおぼすて大将じかたは
うちてたくかれなる御ありさまをいとしかり給て
いとはゞかりにつけてもたうやはらうらめからず
思ひわづらふらんなど覚すやにがあなりやん
じたのためがうきこえ又まりしづのこりに。かげ。
うちずさへなんが
なう＋＋にがたくし給にて
より＋＋にやおぼし給君は。て
かきあふ聞ひめつらう
せられんかぎりな
がにみ奉らせ給いまかく
おりすさでなてゝごろなる
こゝし身ひまりをまりて
かのはしずらへたて
まつり

めえつるはむつゝしくたてゝみんかぎり思はべさむね
がたきそひさだめ。御よきごくりのみこたちもんたど
おをしみこはたはおことまかでられてかふやつくをち
人まつりと思めなどゝみたてまつらんとりぬ御
ことをなん程人をもの二さじなりしとみ奉り
給さるの御あらき気色にきおかしなりぬをいとだにをのどか
してなりとひられ奉り給ひかまあまべなりに
やべうえたる夜もをのめのさまなくつどきこ
ひろ＋＋はくうきいらせ給へりければいかゞおぼゆ。
日の御うはぎしうま々しりはゞかりしうまたけうり
きこえさせずなどに。人にこしのことの＋＋とうたき
もの思ひはてないづさしもんにしも夢のこと
こそなりならせ人侍しで。よろづにてきこえかそ
侍れ。あさましうおぼせど。んにし給さるな
Epoch 12/60
184651/184651 [==============================] - 218s 1ms/step - loss: 2.6907

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "小萩
がもとを思ひこそやれとあれどえ見給はてずいの
ちながさのいとつらうおもふ給"
小萩
がもとを思ひこそやれとあれどえ見給はてずいの
ちながさのいとつらうおもふ給へりいとゞしろ
めとなき給ふ。かくしもみだれ給はんと。みなみだ
つかひなきことなどのみこそは。かくらうたげに
思ひしりて。こゝろことなくて。あまたの御ことを
さしもあらず。おほかたの御ことを。いとおしく
おぼすに。かくては。いとおしうなりぬべきことを
おぼしめぐらみに。おほかたのことをとりたるに。
こゝろのうちに。あやしきことなり。いといたうおぼ
けんとおもひつゞけらるゝことに。あはれなること
などのみこそ。ことなりと。いとかたよりもなくて
み給ふに。ことにかくもてなして。いとかたくも。
いとおかしきことにて。こゝろのうちに。さらに
ことなりしかば。いとおしくて。いとおしくおぼ
しめみるに。こゝろのうちに。いとおしくて。さらに
なん。いとおしくて。あはれにかくことなどこそと。
いとかたはらいたしかば。いとかたくもならず。こと
なる御ことをもみだれたるに。いとなんおぼし
かへり給はんとて。さ
----- diver

おぼしなん思ひとらせ給て。こゝろづかひのすぢも
なり。いたくなん。たゞかの御かたの御はかぎりに。
ものなしなどの給ふ。あやしくおぼしあねて。をの
御こともとなりしかば。かゝることゞもをみそ
まれたるを。うちつゞき給へる。たゞ
----- diversity: 1.0
----- Generating with seed: "ことかは二条のうへのさばかり限にて
なく＋＋申給ときゝしを。いみじきことにおぼ
"
ことかは二条のうへのさばかり限にて
なく＋＋申給ときゝしを。いみじきことにおぼ
しられんことをとり給へばつまどの御めどもをいだてゝ
なきことにはめぐりてあひ君はたちもこんならず
いはせ給ふつらけやかなくの給御はいとまぶきに
こそとても。奉りて。御ばかりを給て。きゝも。ことの
もてなされて。ふかくものし給めることたちいとくち
はなをよりのうちふもえときこえしらぬ世にいより
の御こなすつもりをはかかしたりとてはうたにて。
おぼしこまやらんとはうちすけするに物をげに。
ふるのはやうなりとし世の心もとなしういみじき
は。もてないあれど。ばかりにやむごとなくすみ出ら
れ給に。覚えさせ給など。なをふるかのむすめふ奉り
はづかしきかたにこそあめれど。げんを
。いよ＋＋なを給へなどして。たゞいとはすみだちて人の
おらのたかよりあるさまに。おさやかなり。身もをもひと
もすこしゆやにもすこしむねうたふこめてつゝ御
をしらせ給てもしろしのふらなどふかほれば。
そののどかなるつきなどおほゆさ
----- diversity: 1.2
----- Generating with seed: "ことかは二条のうへのさばかり限にて
なく＋＋申給ときゝしを。いみじきことにおぼ
"
ことかは二条のうへのさばかり限にて
なく＋＋申給ときゝしを。いみじきことにおぼ
うねておなじろくあぶかれ。かろもする物かたの
なるむつべかひて心なざさまなつかしけるしづ
めてきこえけんうしろみ中し程にとりのほど
などゞめのさまよりものなく。この御はとのがさに。すく
そなど。つるみげにいひそゞきことももし侍ける。
人のつれならず。心ゆくやみぬやう中将心ばせ々と
きこゆればめしざらにもあそびも人の御げまの＋＋
などそ。とよしおほきし給御なかせぬ。らすきらんつうめ
も。なをしなたりにつけたゞををしわざらひ給ひ
ぬるよしこと宮

...

しうとのゝひと＋＋かずもなうてならひ給へり
ばかりの御ことをば。なつかしうきこゆ。ましてかた
りねば。ものゝときこゆるにかたじけなくなん
としてうち＋＋の御かたは。あさましうもてなし
給て。それはしものゝつねになきまで。おやかなり
けり。わがおぼしけるこゝちして。けふはかぎりなき
よりも。さらにおもほしける。たゞいかでさせ給て。
かくてこそとめしそひて。ことはりなりと。うと
くだりなどし給ふとて。しり給へる御こゝろには
御かへりさせ給しものを。こゝろにもあらぬを。
すぐれており＋＋にみゆるは。くちなきおとゞも
いでゝ。さだめがたうのみのおやにおとゞにて
。心やすからぬ御けしきなれば。あはれにかはし
たるを。またいとくちなくやと。いとおもしろく。
思ひやりつゝ。くちおしきことゞもを。かくもの
したてもて。したしく。いまなりて。この人の御こゝろ
もなく。かくさま＋＋＄なりと思ひなんところに。
したしきわたりにて。おほとか
----- diversity: 1.0
----- Generating with seed: "の
御こゝろばへ。とりかへしつべうみゆ。いといかめ
しうとのゝひと＋＋かずもなう"
の
御こゝろばへ。とりかへしつべうみゆ。いといかめ
しうとのゝひと＋＋かずもなう思はせんと成
侍るを。いたうわづらはしうしのびて。みえり給
へる物などなりて。よそしふしひ空にいたうはあ
らかむべしと。すざたと思はんこときこしめさど。
なを思ゝれんなに道なぎひ。てもなべておとゆ
へたればゑじらゝつむまいれとうしろめたううちつろ
かめなりと。おとゞもこゆかろかにかたかつらう
思ふべきにや。わづらはしとのしうゝおぼす。かく
はならしきまたかゑなきあそびの御うしろみに。
みることのつよとゞいはんことにも。むかしの御
かへりけるをかぎり身にたてゝ。ひとくつくそへりて。御
はれ給にいとゞかは人え侍らずぞおぼしたりと
の給ひしらんは。なに人にまほしくおぼされるに。
うつくぞ中りてしりけり。ぎたれど。かく右近
いとしくおみがしともおほかりはいづかたの
きこえさせよるをと出てとめずきこえ
かしこがた
にほの
ありつゝ。けうばかりひまか
なと思つゝかたなりしかどほどの
をとらへ宮のも御い
----- diversity: 1.2
----- Generating with seed: "の


りと＋＋みてまかでてあさなきと覚しあこぬ
かたをだにわた
るべかなんと。をよほすこのまぎ聞て。
世にをしいひらひなどなる身ならひばみじだに
おつるなどうち院のみかけ
Epoch 50/60
184651/184651 [==============================] - 211s 1ms/step - loss: 2.5818

----- Generating text after Epoch: 49
----- diversity: 0.2
----- Generating with seed: "。はゞかり
はぢて。みさほにもてつけてみなるゝまゝに。心もげし
うはあらず侍しか"
。はゞかり
はぢて。みさほにもてつけてみなるゝまゝに。心もげし
うはあらず侍しかど。いとおしくおぼさるれば。
いとおかしげに。いとおかしと思ひ給へ。いとおし
くて。あやしき御ことをも。いとかくしたてまつり
給へるを。いとおかしげにて。いとおかしげにて。
いとおしくおぼさるゝ人にいとおしからんと
の給へばいとおしくおぼさるゝにおひ出給へり。
いとおかしげに。いとおしくおぼさるゝにかゝる
御ありさま。いまはじめのうちにもいとおもひ
かし。いとおしく。いまはとて。いとあはれになん。
いかでかたちなど。かゝる御ことを。いとかしこき
ことを。いとおかしげにて。いといたうおぼさるゝに
かたらひて。あながちにいとおしく。いとおかし
きことも。いとおかしう。さらにおはします。たゞ
ひたるさまに。いとおかしげに。いとかしこきに
かゝりて。いといたうおぼさるゝに。いとおしくおも
ひきこえ給へば。いとかしこにて。いとおしくおぼさる
べきことにて。御かた＋＋＄の御ことを。いとおかしき
ことを
----- diversity: 0.5
----- Generating with seed: "。はゞかり
はぢて。みさほにもてつけてみなるゝまゝに。心もげし
うはあらず侍しか"
。はゞかり
はぢて。みさほにもてつけてみなるゝまゝに。心もげし
うはあらず侍しかど。いかでかきつくろひ給ふ。
なにごともあらぬのおとゞの。はかなきことゝ
なきみやの＋＋とし給へど。人々＄も。いとかくして。と
いふもおもしろけん。いまはしかし。つみなくの給
はんと思ひ給へつゝ。めづらしき御心ざしをもし
がほなり。いかにあやしう

す。御ぞどもからろしうこぶべきべきゆきめの。
なにやいとゞみよげならぬ御ざえをしはならぬ
こともかしこちるを。ゆふりおぼすにこゝろをきこえ
えつゝ。のみ所＄給へる＋＋御みは
おほかろ
かずとの。物べき
所あれるわざ
なればおほかたのとかく
てとしへなしなどにおもり
給へばころなるさかりの
御有様ながせはこと
げにおぼすゆづにより
ゆへをしけり。みかどの
御らん共につゐのちゝ
かりもうちろひゞとねをとみ
給ふとはむつましげにおはす
れば上達部は
うきつれ
なる。つら＋＋と。かへりみたて
まつる出たみ。あつかひしらるべし。我すごもにて。どめ
しくおぼしげむしさましてことなびかなどはれし
けりみそのはゝ成けぬしとぞやう＋＋人にはあらねど。
たゞふなん。侍らんこと。こなたに。おりもてならでこ
まめひはなちれなどおはす。ひたいとしをわたらんと
なんと。御君は世人のあさましかば。なにかはと侍を
聞えとをきたりおもひ
----- diversity: 1.2
----- Generating with seed: "りてしげうまがへば。
われにもあらでぬりごめにをしいれられておは
す。御ぞどもか"
りてしげうまがへば。
われにもあらでぬりごめにをしいれられておは
す。御ぞどもからはての給へばいでかびあなりしさ
からることは覚えべをまとつかざみ給てながしきがぼ
らは侍らけぬ。これみつこきながら。ふかく思たぐひつれば
物したなければいふよからはし
給はけづをられ
給うせずおほく
心ふかうはまも
なくくちおしみ手
思ひのゝうちにを
成給たるやうなる
ねはこんもこゝろしねとみえ
給つばり物思ひたまさまいみ
いみじと思ひ
もほのめし給へる
をしそれだて出給ろば
おとゞの給ふしたるに
うつくゝけまやれに
あうつくりつかま少ぞ
とかくすきたり
給ふせてにはてなわらは
しみすこをけんこそかたそ
えりたるけはひを二条
のつゝかなることを
なく心給へられておはす
こたらんといへばこへり
のいざこも心しる
夜いとたろされぬぞや
うをもこたて。人ならば
こず思ひみだれ給やを。え
ゝ聞えそふべじいみた
りわが人ほればむくり
よきくすきてもて物
ひくまふこげましさは
哀かなき御るひよげにひ
Epoch 54/60
184651/184651 [==============================] 

なくて。うちなき給て。かやうにて。なにごとも
まして。御こゝろにもあらず。さるまじきことなり。
いとおかしげにおぼしなげくおはしましたるを
その人のこゝろにもあらぬことを。いとおかしげに
おぼしなげくに。さま＋＋＄におぼしなげく。さま＋＋＄に
おぼして。いとおかしげに。いとおかしげにて。
みなみのたいのつけでに。御かたちなどもなど
いとおしうおぼしなげくに。さるまじきことゝ
なき給へば。いかにおぼしなげ
----- diversity: 0.5
----- Generating with seed: "。おきより舟どものうたひのゝしりて。こぎ
行などもきこゆ。ほのかにたゞちいさき鳥"
。おきより舟どものうたひのゝしりて。こぎ
行などもきこゆ。ほのかにたゞちいさき鳥の
からみにもあらぬひといとさらに。さしはまれ
給けるをいとおかしげにありしかしのかみのため
かしことにてもならひ給はで給て御らんじ
しかゝづらひておはしましけるかのあまり
さりともおもひとりては。とふらひ聞え給へるに。
かやうに思ひかはし給へるを。いとながきふぎの
ついでに。いとおかしげにおぼえ給て。めざましき
かたにこそあれど。すぐしつゝ。かたちなるいと
かくはうちに。さるべき御なをしと。うしろめたう。
かうしかどなんて。とし比＄さかせたまはず。人のおもひ
しらず。人々＄も。したなりけるを。うちのたまへり。
いかに思ひなし。いとおしくおぼして。さる
べきことに。ときこゆるさまにもあらず。めづらし
きことゝ。おかしきことなれば。人のおもひのしなし
けれ。すみつかうまつるとの給ふは。なにごとに
かなしき御ありさまなり。すぐしつゝ。なにごとに
はきゝたまはずなりにけり。みなみのうちにも
かき
----- diversity: 1.0
----- Generating with seed: "。おきより舟どものうたひのゝしりて。こぎ
行などもきこゆ。ほのかにたゞちいさき鳥"
。おきより舟どものうたひのゝしりて。こぎ
行などもきこゆ。ほのかにたゞちいさき鳥の
君ものみあたりて。そむさまなん。こゝろそりきすれわづ。
ひきのゆてものつゝさりつくしなるつゐでにわたり
給ていと聞ゆく心ゐなげなり。ものゝげざきれば。
うつてたらぬはゝ心思しとおぼえいとわりなしき
わざとかたちための臺におはすればつゞく
おとゞもげにあや
し

こなん／こふるがぼ。あやしつ／おことなりすはしめや
なん思ふまりし。こがめきるにてこと＋＋しく。よそ
ふりつるさまはしいに月はたゞ御ずもまたぼらま
ほしうして。かくはしらがりもし出きり給ふなど。か
うたほかばいとゞまつまなんおやのしこしるころ
きしわかげたるいとおしく申さはねどあがらんさま
なはぐかたどきもみえつりをを思ける。へるにをやも
たるくとづらすさかりなりなきて。いかさぞみ奉り
給てじすて殿のやらん。いとにぬがよとりふすましも
したゝく人をのかがしくなん思も参り給おさなし
なんかしとけのたばめと心くまじきおればかたわざし
のたまふうはみこさがなをおぼし世れ。かゝること
しとゝみしろめたかしうなりのね聞ずまいりたり
とふどあも風＋＋などしつゝふたむくせざしうけと
あらはしまじと聞え給のは侍るを。ぬゝむまれども。え
はなちてゆかりきすおそろしくことなる御こゝろも
にのみおはします。いとやみぬ＋＋＄す
